In [1]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta

In [2]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

In [3]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [4]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [5]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [6]:
infy_df = fetchOHLC('INFY','5minute',5)
infy_df.to_csv('infy.csv')

In [7]:
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-06-05 09:15:00+05:30,1400.15,1418.10,1400.15,1409.45,433610
2024-06-05 09:20:00+05:30,1409.00,1412.65,1404.00,1404.90,290268
2024-06-05 09:25:00+05:30,1404.95,1406.00,1401.95,1405.00,1064215
2024-06-05 09:30:00+05:30,1405.00,1410.05,1403.10,1404.90,321496
2024-06-05 09:35:00+05:30,1404.90,1408.95,1403.05,1404.80,150415


In [8]:
def doji(df):
    df = df.copy()
    avg_candle_size = abs(df['close'] - df['open']).median()
    df['doji'] = abs(df['close'] - df['open']) <= (0.05*avg_candle_size)
    return df

In [9]:
doji_df = doji(infy_df)
doji_df


,open,high,low,close,volume,doji
date,,,,,,
2024-06-05 09:15:00+05:30,1400.15,1418.10,1400.15,1409.45,433610,False
2024-06-05 09:20:00+05:30,1409.00,1412.65,1404.00,1404.90,290268,False
2024-06-05 09:25:00+05:30,1404.95,1406.00,1401.95,1405.00,1064215,True
2024-06-05 09:30:00+05:30,1405.00,1410.05,1403.10,1404.90,321496,False
2024-06-05 09:35:00+05:30,1404.90,1408.95,1403.05,1404.80,150415,False
...,...,...,...,...,...,...
2024-06-10 09:15:00+05:30,1525.30,1529.80,1516.00,1516.95,439043,False
2024-06-10 09:20:00+05:30,1516.75,1517.40,1514.20,1516.40,279808,False
2024-06-10 09:25:00+05:30,1516.80,1516.95,1514.15,1514.15,111061,False
